# Setup Notebook

In [1]:
# When working in AI LRZ
%cd ~/cma/CMA_Fairness_v2

/dss/dsshome1/0C/ra93lal2/cma/CMA_Fairness_v2


/dss/dsshome1/0C/ra93lal2/.local/share/virtualenvs/CMA_Fairness_v2-3j10GkSs/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/dss/dsshome1/0C/ra93lal2/.local/share/virtualenvs/CMA_Fairness_v2-3j10GkSs/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
%run analysis_setup_cp.ipynb

/dss/dsshome1/0C/ra93lal2/cma/CMA_Fairness_v2
The data has N = 100 rows and N = 12 columns.


## Prepare Data


In [ ]:
# Filter out a single configuration to keep data manageable
#df_agg = df_agg_full[
#  (df_agg_full["sett_eval_fairness_grouping"] == "nationality-all") #&
#  #(df_agg_full["sett_eval_exclude_subgroups"] == "keep-in-eval") &
#  #(df_agg_full["sett_eval_on_subset"] == "full")
#][
#  # Remove eval columns
#  cols_non_eval + cols_performance + cols_fairness
#].reset_index(drop = True)
#
#df_agg.shape

In [3]:
import pandas as pd
from pathlib import Path

RUN_TO_ANALYSE = "9"
CP_DIR = Path("output") / "runs" / str(RUN_TO_ANALYSE)

df_agg = pd.read_csv(CP_DIR / "combined_cp_metrics.csv")

In [4]:
df_agg.head()

,universe_id,universe_model,universe_exclude_features,universe_exclude_subgroups,universe_scale,q_hat,coverage,avg_size,cov_frau1,cov_nongerman,cov_nongerman_male,cov_nongerman_female
0,4356e83194008df6152c133bfc881349,penalized_logreg,nationality-sex,keep-all,scale,0.661153,0.911415,1.124501,0.912144,0.910627,0.925335,0.886289
1,41eaf70782011ea35bf8303122f41955,elasticnet,none,drop-non-german,do-not-scale,0.685251,0.906644,1.135024,0.905432,0.906978,0.922183,0.881818
2,60eeda26081f16b3f8f6849c5e95d393,penalized_logreg,nationality-sex,drop-non-german,do-not-scale,0.669111,0.911727,1.211716,0.911659,0.935160,0.946681,0.916095
3,4174901d0e4d984240c819a4abc731b9,gbm,none,drop-non-german,scale,0.658725,0.911459,1.110645,0.914571,0.909560,0.925245,0.883607
4,22c2dffeab3d2bb5c3ec86fe0972c853,logreg,nationality,drop-non-german,scale,0.660298,0.911470,1.124746,0.911417,0.910459,0.925786,0.885097


In [5]:
df_agg.shape

(100, 12)

In [6]:
main_cp_metric = "avg_size" # "avg_size", "cov_nongerman_female"

In [ ]:
#import matplotlib.pyplot as plt
#import seaborn as sns
#
## Plot distribution of q_hat
#plt.figure(figsize=(8, 5))
#sns.histplot(df_agg["q_hat"], bins=10, kde=True, color="skyblue")
#plt.title("Distribution of q_hat Values")
#plt.xlabel("q_hat")
#plt.ylabel("Frequency")
#plt.grid(True)
#plt.tight_layout()
#plt.show()

# Calculate Variable Importance

## Use a Lasso Regression to estimate Importance of Settings

In [7]:
X = df_agg[["universe_model", "universe_exclude_features", "universe_exclude_subgroups", "universe_scale"]]
y = df_agg[main_cp_metric]

In [8]:
X.head()

,universe_model,universe_exclude_features,universe_exclude_subgroups,universe_scale
0,penalized_logreg,nationality-sex,keep-all,scale
1,elasticnet,none,drop-non-german,do-not-scale
2,penalized_logreg,nationality-sex,drop-non-german,do-not-scale
3,gbm,none,drop-non-german,scale
4,logreg,nationality,drop-non-german,scale


In [9]:
X.shape

(100, 4)

In [10]:
y.shape

(100,)

### Main Effects of Settings Only (i.e. no interactions)

In [11]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LassoCV

# Do preprocessing in a separate pipeline from model fitting
# (for eli5 to work)
preprocessor = make_pipeline(
    OneHotEncoder(), # Converts categorical features to one-hot encoded format
).fit(X) # Fit the preprocessing pipeline to the input data X
X_processed = preprocessor.transform(X) # This creates a numeric matrix suitable for modeling

# Fit a Lasso regression model with cross-validation to the processed data
lasso_reg = LassoCV(cv=5, random_state=0).fit(X_processed, y)

# Check whether it's predictive at all
lasso_reg.score(X_processed, y) # Evaluate the model's R^2 score on the training data


0.6125997485294356

R² = 1.0: Perfect predictions

R² = 0.0: Model does no better than predicting the mean

R² < 0.0: Model is worse than just predicting the mean

Weights of importance:

In [12]:
# Visualize the importance of each feature in trained lasso_reg model
import eli5
eli5.show_weights(lasso_reg, top=-1, feature_names = preprocessor.get_feature_names_out())


Weight?,Feature
+1.120,<BIAS>
+0.040,universe_scale_do-not-scale
+0.024,universe_model_penalized_logreg
+0.024,universe_model_logreg
+0.015,universe_exclude_features_age
-0.000,universe_scale_scale
-0.028,universe_model_gbm


### Including Interactions

In [13]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LassoCV

# Do preprocessing w/o a pipeline for eli5 to work
preprocessor = make_pipeline(
    OneHotEncoder(),
    PolynomialFeatures(degree=2),
).fit(X)
X_processed = preprocessor.transform(X)

lasso_reg = LassoCV(cv=5, random_state=0).fit(X_processed, y)

# Check whether it's predictive at all
lasso_reg.score(X_processed, y) # Evaluate the model's R^2 score on the training data


0.9538009860852997

Weights of importance:

In [14]:
import eli5
eli5.show_weights(lasso_reg, top=-1, feature_names = preprocessor.get_feature_names_out())


Weight?,Feature
+1.126,<BIAS>
+0.068,universe_model_penalized_logreg universe_scale_do-not-scale
+0.068,universe_model_logreg universe_scale_do-not-scale
+0.036,universe_exclude_features_age universe_scale_do-not-scale
+0.027,universe_model_elasticnet universe_exclude_features_age
+0.016,universe_model_rf universe_scale_scale
+0.013,universe_scale_do-not-scale
+0.004,universe_exclude_subgroups_drop-non-german universe_scale_do-not-scale
-0.000,universe_scale_scale
-0.001,universe_model_rf universe_scale_do-not-scale


## Use a functinoal ANOVA (fANOVA) to Analyze Setting Importance

Based on the following paper:

Hutter, F., Hoos, H., & Leyton-Brown, K. (2014). An Efficient Approach for Assessing Hyperparameter Importance. Proceedings of the 31st International Conference on Machine Learning, 754–762. https://proceedings.mlr.press/v32/hutter14.html


In [15]:
from fairness_multiverse.analysis import MultiverseFanova

m_fanova = MultiverseFanova(features = df_agg[["universe_model", "universe_exclude_features", "universe_exclude_subgroups", "universe_scale"]], outcome = df_agg[main_cp_metric])


In [16]:
m_fanova.quantify_individual_importance()

,individual importance,total importance,individual std,total std
universe_exclude_features,0.052626,0.052626,0.038208,0.038208
universe_exclude_subgroups,0.007430,0.007430,0.017000,0.017000
universe_model,0.265678,0.265678,0.059307,0.059307
universe_scale,0.259512,0.259512,0.080380,0.080380


In [17]:
m_fanova.quantify_importance(save_to = "fanova_importance_interactions-overall_alpha_2.csv")

,level_0,level_1,level_2,level_3,individual importance,total importance,individual std,total std
2,universe_model,NaN,NaN,NaN,0.265678,0.265678,0.059307,5.930683e-02
3,universe_scale,NaN,NaN,NaN,0.259512,0.259512,0.080380,8.038002e-02
9,universe_model,universe_scale,NaN,NaN,0.242744,0.767934,0.072518,1.084481e-01
5,universe_exclude_features,universe_model,NaN,NaN,0.053127,0.371431,0.033196,6.281583e-02
0,universe_exclude_features,NaN,NaN,NaN,0.052626,0.052626,0.038208,3.820757e-02
12,universe_exclude_features,universe_model,universe_scale,NaN,0.037320,0.938795,0.032566,8.218643e-02
6,universe_exclude_features,universe_scale,NaN,NaN,0.027788,0.339926,0.022032,9.424135e-02
10,universe_exclude_features,universe_exclude_subgroups,universe_model,NaN,0.012477,0.407084,0.015277,9.296453e-02
14,universe_exclude_features,universe_exclude_subgroups,universe_model,universe_scale,0.012364,1.000000,0.022084,3.132325e-15
7,universe_exclude_subgroups,universe_model,NaN,NaN,0.010588,0.283696,0.026983,8.422689e-02


In [18]:
# Format

cols_to_format = [
    "individual importance",
    "total importance",
    "individual std",
    "total std"
]

fanova_interactions = m_fanova.quantify_importance()
fanova_interactions[cols_to_format] = fanova_interactions[cols_to_format].applymap(lambda x: f"{x:.4f}")
fanova_interactions

,level_0,level_1,level_2,level_3,individual importance,total importance,individual std,total std
2,universe_model,NaN,NaN,NaN,0.2657,0.2657,0.0593,0.0593
3,universe_scale,NaN,NaN,NaN,0.2595,0.2595,0.0804,0.0804
9,universe_model,universe_scale,NaN,NaN,0.2427,0.7679,0.0725,0.1084
5,universe_exclude_features,universe_model,NaN,NaN,0.0531,0.3714,0.0332,0.0628
0,universe_exclude_features,NaN,NaN,NaN,0.0526,0.0526,0.0382,0.0382
12,universe_exclude_features,universe_model,universe_scale,NaN,0.0373,0.9388,0.0326,0.0822
6,universe_exclude_features,universe_scale,NaN,NaN,0.0278,0.3399,0.0220,0.0942
10,universe_exclude_features,universe_exclude_subgroups,universe_model,NaN,0.0125,0.4071,0.0153,0.0930
14,universe_exclude_features,universe_exclude_subgroups,universe_model,universe_scale,0.0124,1.0000,0.0221,0.0000
7,universe_exclude_subgroups,universe_model,NaN,NaN,0.0106,0.2837,0.0270,0.0842


In [ ]:
best_p_margs = m_fanova.fanova.get_most_important_pairwise_marginals(n=5)
print(best_p_margs)

In [ ]:
# Directory that will contain outputs from analysis

from pathlib import Path

ANALYSIS_OUTPUT_DIR = Path(".") / "analyses"
ANALYSIS_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
from fanova import visualizer

vis = visualizer.Visualizer(m_fanova.fanova, m_fanova.configuration_space, directory = str(ANALYSIS_OUTPUT_DIR))

In [ ]:
vis.plot_marginal(0)

In [ ]:
vis.plot_pairwise_marginal(['universe_exclude_features', 'universe_exclude_subgroups'])

## Quantify Importance with Partial Data

In [ ]:
from tqdm import tqdm

PARTIAL_FANOVA_DIR = ANALYSIS_OUTPUT_DIR / "partial_fanova" / "overall"
PARTIAL_FANOVA_DIR.mkdir(parents=True, exist_ok=True)

N_ITERATIONS = 10

In [ ]:
from fairness_multiverse.analysis import MultiverseFanova
import joblib

def quantify_importance_for_fraction(fraction: float, base_directory = PARTIAL_FANOVA_DIR):
    # Get random subset of the data
    df = df_agg.sample(frac = fraction).reset_index(drop = True)
    data_hash = joblib.hash(df)

    # Create directory for this fraction
    directory = base_directory / f"fraction-{fraction}"
    directory.mkdir(exist_ok = True)

    # Run FANOVA on subset
    partial_fanova = MultiverseFanova(features = df[["universe_model", "universe_exclude_features", "universe_exclude_subgroups"]], outcome = df[main_cp_metric])
    partial_fanova.quantify_importance(save_to = directory / f"partial-fanova_importance_interactions-majmin-{fraction}-{data_hash}.csv")

In [ ]:
for i in tqdm(range(N_ITERATIONS)):
    quantify_importance_for_fraction(0.01)

In [ ]:
for i in tqdm(range(N_ITERATIONS)):
    quantify_importance_for_fraction(0.05)

In [ ]:
for i in tqdm(range(N_ITERATIONS)):
    quantify_importance_for_fraction(0.1)

In [ ]:
for i in tqdm(range(N_ITERATIONS)):
    quantify_importance_for_fraction(0.2)